In [ ]:

from random import shuffle
from math import floor
from cv2 import cvtColor, COLOR_BGR2GRAY
import numpy as np
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

def CNN(images, labels, dimension, LR, target):

    MODEL_NAME = 'testing-{}-{}.model'.format(LR, '6conv-basic')
    
    #one-hot encode the labels of the images [b,c,g,h,m]
    encoded_class = list()
    if target == 'Black Spot':
        for each in labels:
            if each == 'Black Spot':
                encoded_class.append([1,0])
            else:
                encoded_class.append([0,1])
    elif target == 'Canker':
        for each in labels:
            if each == 'Canker':
                encoded_class.append([1,0])
            else:
                encoded_class.append([0,1])
    elif target == 'Greening':
        for each in labels:
            if each == 'Greening':
                encoded_class.append([1,0])
            else:
                encoded_class.append([0,1])
    else:
        for each in labels:
            if each == 'Healthy':
                encoded_class.append([1,0])
            else:
                encoded_class.append([0,1])

    #convert images to grayscale to simplify problem
    for index in range(len(images)):
        #print(len(images[index][0][0]))
        images[index] = cvtColor(images[index], COLOR_BGR2GRAY)
   
    #combine labels and images into one list and shuffle
    data = list()
    for index in range(len(images)):
        data.append([np.array(images[index]),np.array(encoded_class[index])])
    shuffle(data)
    
    train_data = data[:550]
    test_data = data[550:]
    
    for index in range(len(test_data)):
        test_data[index][1] = index


    #create layers of neural network
    net = input_data(shape =[None, dimension, dimension, 1], name ='input')
    
    net = conv_2d(net, 32, 5, activation ='relu')
    net = max_pool_2d(net, 5)
  
    net = conv_2d(net, 64, 5, activation ='relu')
    net = max_pool_2d(net, 5)
  
    net = conv_2d(net, 128, 5, activation ='relu')
    net = max_pool_2d(net, 5)
  
    net = conv_2d(net, 64, 5, activation ='relu')
    net = max_pool_2d(net, 5)

    net = conv_2d(net, 32, 5, activation ='relu')
    net = max_pool_2d(net, 5)
  
    net = fully_connected(net, 1024, activation ='relu')
    net = dropout(net, 0.8)
  
    net = fully_connected(net, 2, activation ='softmax')
    net = regression(net, learning_rate = LR, loss ='categorical_crossentropy', name ='targets')
  
    model = tflearn.DNN(net, tensorboard_dir ='log')
    

    # Splitting the testing data and training data
    size = len(train_data)
    split = floor(size*.8)
    train = train_data[:split]
    test = train_data[split:]
    
    # X-Features & Y-Labels
    X = np.array([i[0] for i in train]).reshape(-1, dimension, dimension, 1)
    Y = [i[1] for i in train]
    test_x = np.array([i[0] for i in test]).reshape(-1, dimension, dimension, 1)
    test_y = [i[1] for i in test]
    
    

    # epoch = 5 taken
    model.fit({'input': X}, {'targets': Y}, n_epoch = 5, 
    validation_set =({'input': test_x}, {'targets': test_y}), 
    snapshot_step = 500, show_metric = True, run_id = MODEL_NAME)
    model.save(MODEL_NAME, tensorboard_dir ='log') 
    
    import matplotlib.pyplot as plt
    # if you need to create the data:
    # test_data = process_test_data()
    # if you already have some saved:

    fig = plt.figure()

    for num, data in enumerate(test_data[:20]):
      
        img_num = data[1]
        img_data = data[0]
      
        y = fig.add_subplot(4, 5, num + 1)
        orig = img_data
        data = img_data.reshape(dimension, dimension, 1)
  
        # model_out = model.predict([data])[0]
        model_out = model.predict([data])[0]

        if np.argmax(model_out) == 1: str_label ='Dog'
        else: str_label ='Cat'
          
        y.imshow(orig, cmap ='gray')
        plt.title(str_label)
        y.axes.get_xaxis().set_visible(False)
        y.axes.get_yaxis().set_visible(False)

    plt.show()

    return 0

In [ ]:
black_spot = list()
canker = list()
greening = list()
healthy = list()
melanose = list()

master_images = list() # compiled after resizing to contain images from all sets
master_colors = list() # same as master images but each pixel is a string name for its color instead of a BGR component list
master_file_names = list() # contains the names of the files in their appropriate order
true_class = list() # a list of the true classifications of each image

dimension = 8 # lowered for quicker testing 
LR = 1e-3 #learning rate for CNN

black_spot, canker, greening, healthy, melanose = Resize_Images(dimension) # Resize images and set each of them to be compatible with OpenCV
#print(black_spot[0])
# Get the average representation of each of our identifier colors from the datasets, output is a list representing the average
# presence of that color for the entire class represented by the list. This is in the form of [ green, brown, yellow ]. It also returns
# a list of each image where all the pixels have been converted to their color names
black_spot_color_averages, black_spot_pixels_as_color = BGR_Calculation(black_spot, dimension)
canker_color_averages, canker_pixels_as_color = BGR_Calculation(canker, dimension)
greening_color_averages, greening_pixels_as_color = BGR_Calculation(greening, dimension)
healthy_color_averages, healthy_pixels_as_color = BGR_Calculation(healthy, dimension)
melanose_color_averages, melanose_pixels_as_color = BGR_Calculation(melanose, dimension)

# Combine lists to make a master of the previous datasets in order to split into training and testing data.
Combine_Lists()

#print(master_images[0])
test = CNN(master_images, true_class, dimension, LR, 'Healthy')